In [2]:
import pandas as pd
import numpy as np
from causalimpact import CausalImpact

In [3]:
def custom_sort_key(s):
    parts = s.split('_')
    return int(parts[1])

In [5]:
def causalimpact_eval(dataset_name,dataset_type,forecast_horizon):
    if dataset_type == "sim":
        # y_true_df_A = pd.read_csv("../datasets/text_data/" + dataset_type +  \
        #         "/" + dataset_name + "_test_actual.csv")
        # # Reading the original data to calculate the MASE errors
        # y_true_df_B = pd.read_csv("../datasets/text_data/" + dataset_type +  \
        #         "/" + dataset_name + "_train.csv")
        # data_row_A = y_true_df_A.pivot(index='time', columns='series_id', values='value')
        # data_row_B = y_true_df_B.pivot(index='time', columns='series_id', values='value')
        # data_row = pd.concat([data_row_B, data_row_A],ignore_index=True)
        # data_row_A = data_row_A.T
        # data_row_B = data_row_B.T
        data_row = pd.read_csv("../datasets/text_data/sim/"+dataset_name+".csv")
        length_of_series = len(data_row.index)
        data_row_for_errors = pd.read_csv("../datasets/text_data/sim/"+dataset_name+"_for_errors.csv").iloc[:,1:]
        data_row_A = data_row_for_errors.iloc[length_of_series-forecast_horizon:, :].T
        data_row_B = data_row_for_errors.iloc[:length_of_series-forecast_horizon, :].T

    if dataset_type == "calls911":
        control = ["BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO",
                      "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD",
                      "SPRINGFIELD", "TRAPPE"]
        data_row = pd.read_csv('../datasets/text_data/' + dataset_type\
                            + '/'+dataset_name+'.csv').iloc[:, 1:]
        # y_true_df_A = data_row.iloc[len(data_row['date'])-forecast_horizon:, 1:].T
        # y_true_df_B = data_row.iloc[:len(data_row['date'])-forecast_horizon, 1:].T
        # data_row_A = y_true_df_A
        # data_row_B = y_true_df_B
        data_row_cols = data_row.columns
        data_row_for_errors = data_row.loc[:,control]
        length_of_series = len(data_row.index)
        y_true_df_A = data_row_for_errors.iloc[length_of_series-forecast_horizon:, :].T
        y_true_df_B = data_row_for_errors.iloc[:length_of_series-forecast_horizon, :].T
        data_row_A = y_true_df_A
        # print(data_row_A)
        data_row_B = y_true_df_B
    seasonality_period = 12
    errors_directory = '../results/benchmarks/errors/'

    errors_file_name_mean_median = 'mean_median_' + dataset_name + '_causalimpact'
    SMAPE_file_name_all_errors = 'all_smape_errors_' + dataset_name + '_causalimpact'
    MASE_file_name_all_errors = 'all_mase_errors_' + dataset_name + '_causalimpact'

    errors_file_full_name_mean_median = errors_directory + errors_file_name_mean_median+'.txt'
    SMAPE_file_full_name_all_errors = errors_directory + SMAPE_file_name_all_errors
    MASE_file_full_name_all_errors = errors_directory + MASE_file_name_all_errors
    
    output = '../results/benchmarks/predicted/' + dataset_name +\
          '_causalimpact.csv'
    y_pred_list = []
    for i in data_row.columns:
        ci = CausalImpact(data_row.loc[:,[i] + [col for col in \
                    data_row.columns if col != i]],
                [0,length_of_series-forecast_horizon-1],
                [length_of_series-forecast_horizon,
                length_of_series-1], nseasons=[{'period': seasonality_period}])
        # evaluate the model
        y_pred = ci.inferences.loc[(length_of_series-\
                    forecast_horizon):(length_of_series-1),'preds']
        y_pred_list.append(y_pred)
    y_pred_df = pd.DataFrame(y_pred_list)
    y_pred_df.to_csv(output, index=False, header=False)
    # np.savetxt(output, pd.DataFrame(y_pred_list), delimiter = ',')

    # y_pred_df= pd.read_csv(output, header=None)
    y_pred_for_errors = y_pred_df.copy()
    if dataset_type == "calls911":
        y_pred_for_errors['names'] = data_row_cols
        y_pred_for_errors.set_index('names', inplace=True)
        y_pred_for_errors = y_pred_for_errors.loc[control,:]
    
    no_of_series = len(data_row_B.index) 

    # SMAPE
    time_series_wise_SMAPE = 2 * np.abs(y_pred_for_errors - np.array(data_row_A)) /\
        (np.abs(y_pred_for_errors) + np.abs(np.array(data_row_A)))
    SMAPEPerSeries = np.mean(time_series_wise_SMAPE, axis=1)
    mean_SMAPE = np.mean(SMAPEPerSeries)
    mean_SMAPE_str = f"mean_SMAPE:{mean_SMAPE}"
    print(mean_SMAPE_str)
    np.savetxt(SMAPE_file_full_name_all_errors+'.txt', SMAPEPerSeries, delimiter=",", fmt='%f')
    
    mase_vector = []
    for i in range(no_of_series):
        lagged_diff = [data_row_B.iloc[i,j] - \
                   data_row_B.iloc[i,j - seasonality_period]\
                      for j in range(seasonality_period,\
                        len(data_row_B.columns))]
        mase_vector.append(np.mean(np.abs(np.array(np.array(data_row_A.iloc[i]))\
                 - np.array(y_pred_for_errors.iloc[i])) / np.mean(np.abs(lagged_diff))))

    mean_MASE = np.mean(mase_vector)
    mean_MASE_str = f"mean_MASE:{mean_MASE}"
    print(mean_MASE_str)

    np.savetxt(MASE_file_full_name_all_errors+'.txt', mase_vector, delimiter=",", fmt='%f')

    # Writing the SMAPE results to file
    with open(errors_file_full_name_mean_median, 'w') as f:
        # f.write('\n'.join([mean_SMAPE_str, median_SMAPE_str, std_SMAPE_str]))
        f.write('\n'.join([mean_SMAPE_str]))

    # Writing the MASE results to file
    with open(errors_file_full_name_mean_median, 'a') as f:
        # f.write('\n'.join([mean_MASE_str, median_MASE_str, std_MASE_str]))
        f.write('\n'.join([mean_MASE_str]))


In [4]:
dataset_name = 'calls911_benchmarks'
dataset_type = 'calls911'
forecast_horizon=7

control = ["BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO",
                      "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD",
                      "SPRINGFIELD", "TRAPPE"]
data_row = pd.read_csv('../datasets/text_data/' + dataset_type\
                    + '/'+dataset_name+'.csv').iloc[:, 1:]
# y_true_df_A = data_row.iloc[len(data_row['date'])-forecast_horizon:, 1:].T
# y_true_df_B = data_row.iloc[:len(data_row['date'])-forecast_horizon, 1:].T
# data_row_A = y_true_df_A
# data_row_B = y_true_df_B
data_row_cols = data_row.columns
data_row_for_errors = data_row.loc[:,control]
length_of_series = len(data_row.index)
y_true_df_A = data_row_for_errors.iloc[length_of_series-forecast_horizon:, :].T
y_true_df_B = data_row_for_errors.iloc[:length_of_series-forecast_horizon, :].T
data_row_A = y_true_df_A
# print(data_row_A)
data_row_B = y_true_df_B

seasonality_period = 12
errors_directory = '../results/benchmarks/errors/'

errors_file_name_mean_median = 'mean_median_' + dataset_name + '_causalimpact'
SMAPE_file_name_all_errors = 'all_smape_errors_' + dataset_name + '_causalimpact'
MASE_file_name_all_errors = 'all_mase_errors_' + dataset_name + '_causalimpact'

errors_file_full_name_mean_median = errors_directory + errors_file_name_mean_median+'.txt'
SMAPE_file_full_name_all_errors = errors_directory + SMAPE_file_name_all_errors
MASE_file_full_name_all_errors = errors_directory + MASE_file_name_all_errors

output = '../results/benchmarks/predicted/' + dataset_name +\
        '_causalimpact.csv'
y_pred_list = []
i = data_row.columns[0]
ci = CausalImpact(data_row.loc[:,[i] + [col for col in \
            data_row.columns if col != i]],
        [0,length_of_series-forecast_horizon-1],
        [length_of_series-forecast_horizon,
        length_of_series-1], nseasons=[{'period': seasonality_period}])
# evaluate the model
y_pred = ci.inferences.loc[(length_of_series-\
            forecast_horizon):(length_of_series-1),'preds']
y_pred_list.append(y_pred)

/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: standardize, nseasons. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [17]:
np.mean(y_pred_list)

625.5709418160271

In [15]:
print(ci.summary('report'))

Analysis report {CausalImpact}


During the post-intervention period, the response variable had
an average value of approx. 604.0. By contrast, in the absence of an
intervention, we would have expected an average response of 625.57.
The 95% interval of this counterfactual prediction is [623.45, 627.52].
Subtracting this prediction from the observed response yields
an estimate of the causal effect the intervention had on the
response variable. This effect is -21.57 with a 95% interval of
[-23.52, -19.45]. For a discussion of the significance of this effect,
see below.


Summing up the individual data points during the post-intervention
period (which can only sometimes be meaningfully interpreted), the
response variable had an overall value of 4228.0.
By contrast, had the intervention not taken place, we would have expected
a sum of 4379.0. The 95% interval of this prediction is [4364.15, 4392.63].


The above results are given in terms of absolute numbers. In relative
terms, the respons

In [13]:
print(ci.summary())

Posterior Inference {Causal Impact}
                          Average            Cumulative
Actual                    604.0              4228.0
Prediction (s.d.)         625.57 (1.04)      4379.0 (7.26)
95% CI                    [623.45, 627.52]   [4364.15, 4392.63]

Absolute effect (s.d.)    -21.57 (1.04)      -151.0 (7.26)
95% CI                    [-23.52, -19.45]   [-164.63, -136.15]

Relative effect (s.d.)    -3.45% (0.17%)     -3.45% (0.17%)
95% CI                    [-3.76%, -3.11%]   [-3.76%, -3.11%]

Posterior tail-area probability p: 0.0
Posterior prob. of a causal effect: 100.0%

For more details run the command: print(impact.summary('report'))


In [5]:
ci.inferences

,post_cum_y,preds,post_preds,post_preds_lower,post_preds_upper,preds_lower,preds_upper,post_cum_pred,post_cum_pred_lower,post_cum_pred_upper,point_effects,point_effects_lower,point_effects_upper,post_cum_effects,post_cum_effects_lower,post_cum_effects_upper
48,0.0,796.029014,NaN,NaN,NaN,794.671731,797.386297,0.000000,0.000000,0.000000,-0.029014,-1.386297,1.328269,0.000000,0.000000,0.000000
49,693.0,648.615465,648.615465,647.294302,649.936628,647.294302,649.936628,648.615465,647.394315,649.934618,44.384535,43.063372,45.705698,44.384535,43.065382,45.605685
50,1332.0,733.832294,733.832294,731.979793,735.684795,731.979793,735.684795,1382.447759,1379.627729,1385.343802,-94.832294,-96.684795,-92.979793,-50.447759,-53.343802,-47.627729
51,1907.0,562.704477,562.704477,560.455282,564.953673,560.455282,564.953673,1945.152236,1940.572623,1949.607279,12.295523,10.046327,14.544718,-38.152236,-42.607279,-33.572623
52,2425.0,628.397044,628.397044,625.822780,630.971308,625.822780,630.971308,2573.549280,2566.737777,2579.892314,-110.397044,-112.971308,-107.822780,-148.549280,-154.892314,-141.737777
53,2929.0,558.100643,558.100643,555.248342,560.952944,555.248342,560.952944,3131.649923,3122.193937,3140.482968,-54.100643,-56.952944,-51.248342,-202.649923,-211.482968,-193.193937
54,3614.0,770.906627,770.906627,767.810624,774.002629,767.810624,774.002629,3902.556550,3890.283148,3913.544896,-85.906627,-89.002629,-82.810624,-288.556550,-299.544896,-276.283148
55,4228.0,476.440043,476.440043,473.127086,479.753000,473.127086,479.753000,4378.996593,4364.154313,4392.626805,137.559957,134.247000,140.872914,-150.996593,-164.626805,-136.154313
0,NaN,571.204953,NaN,NaN,NaN,-377766.337750,378908.747656,NaN,NaN,NaN,-57.204953,-378394.747656,378280.337750,NaN,NaN,NaN
1,NaN,781.666797,NaN,NaN,NaN,-377555.875908,379119.209502,NaN,NaN,NaN,-54.666797,-378392.209502,378282.875908,NaN,NaN,NaN


In [6]:
dataset_name = 'calls911_benchmarks'
dataset_type = 'calls911'
forecast_horizon=7
causalimpact_eval(dataset_name,dataset_type,forecast_horizon)

/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score.

mean_SMAPE:0.4605758534262359
mean_MASE:1.6627558891061813


In [4]:
dataset_name_test = ['sim_10_60_l_he', 'sim_10_60_l_ho',\
                     'sim_10_60_nl_he', 'sim_10_60_nl_ho',\
                     'sim_10_222_l_he', 'sim_10_222_l_ho',\
                     'sim_10_222_nl_he', 'sim_10_222_nl_ho',\
                     'sim_101_60_l_he', 'sim_101_60_l_ho',\
                     'sim_101_60_nl_he', 'sim_101_60_nl_ho',\
                     'sim_101_222_l_he', 'sim_101_222_l_ho',\
                     'sim_101_222_nl_he', 'sim_101_222_nl_ho',\
                     'sim_500_60_l_he', 'sim_500_60_l_ho',\
                     'sim_500_60_nl_he', 'sim_500_60_nl_ho',\
                     'sim_500_222_l_he', 'sim_500_222_l_ho',\
                     'sim_500_222_nl_he', 'sim_500_222_nl_ho']
dataset_type = 'sim'
forecast_horizon=12
for i in dataset_name_test:
    print(i)
    causalimpact_eval(i,dataset_type,forecast_horizon)

sim_10_60_l_he


/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarnin

mean_SMAPE:0.27958661942690627
mean_MASE:0.8977582814708288
sim_10_60_l_ho


/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarnin

mean_SMAPE:0.38903406371272575
mean_MASE:1.2867213326157017
sim_10_60_nl_he


...

mean_SMAPE:0.2536683273817023
mean_MASE:0.9772216651648538
sim_500_222_nl_he


/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score.

mean_SMAPE:0.4741023117202991
mean_MASE:0.8620544756260763
sim_500_222_nl_ho


/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: nseasons, standardize. After release 0.14, this will raise.
  warnings.warn(
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score.

mean_SMAPE:0.472377586486936
mean_MASE:0.8723096485896937
